### Define Inception Model

In [28]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 320, 240
train_data_dir = 'datasets/UCF11/frames/train'
validation_data_dir = 'datasets/UCF11/frames/test'
nb_train_samples = 7365
nb_validation_samples = 1825 
batch_size = 16
epochs = 50
nb_class = 11

model = applications.inception_v3.InceptionV3(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

### Customize Model

In [29]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(nb_class, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

### (Optional) Convert BatchNorm to LocalNorm

In [30]:
from scripts.layers.LocalNorm import LocalNormalization

group_size = 4

for i in range(len(model_final.layers)):
    if type(model_final.layers[i]) == BatchNormalization:
        model_final.layers[i] = LocalNormalization(groupsize=group_size, batch_size=batch_size)

### Freeze First Layers

In [31]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model_final.layers[:5]:
    layer.trainable = False

### Generate Data

In [32]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_width, img_height),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_width, img_height),
class_mode = "categorical")

Found 7365 images belonging to 11 classes.
Found 1825 images belonging to 11 classes.


### Train Model

In [ ]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("inception_v3_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
validation_steps = nb_validation_samples,
callbacks = [checkpoint, early])

/Users/okyksl/miniconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=50, validation_data=<keras_pre..., validation_steps=1825, callbacks=[<keras.ca..., steps_per_epoch=460)`
  app.launch_new_instance()


Epoch 1/50
